# **CS565 - Intelligent Systems and Interfaces - Assignment 2**
#### **Lavish Gulati - 170101082**

In [ ]:
# Read corpus from Google Drive
from google.colab import drive
drive.mount("/content/drive")

import codecs
eng_text = codecs.open('/content/drive/My Drive/CS565/Assignment 2/en_wiki.txt', 'r').read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing required libraries
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from random import shuffle, uniform
import math
import statistics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2.1 - N-gram language model

#### 0 - Data Preprocessing and Splitting

In [ ]:
# Fraction subset of corpus taken
SMALL_RATIO = 1
eng_text = eng_text[0:int(SMALL_RATIO*len(eng_text))]

In [ ]:
# Sentence segmentation
sentences = []
sentences = sent_tokenize(eng_text)
print('Number of sentences in corpus:', len(sentences))
print('Sample data:', sentences[0:5])

Number of sentences in corpus: 761582
Sample data: ['The word "atom" was coined by ancient Greek philosophers.', 'However, these ideas were founded in philosophical and theological reasoning rather than evidence and experimentation.', 'As a result, their views on what atoms look like and how they behave were incorrect.', 'They also could not convince everybody, so atomism was but one of a number of competing theories on the nature of matter.', 'It was not until the 19th century that the idea was embraced and refined by scientists, when the blossoming science of chemistry produced discoveries that only the concept of atoms could explain.']


In [ ]:
# Word tokenization of each sentence
word_tokens = []
word_count = 0
for sentence in sentences:
  words = word_tokenize(sentence)
  word_count += len(words)
  word_tokens.append(words)
print('Number of words in corpus:', word_count)
print('Sample data:', word_tokens[0:10])

Number of words in corpus: 19602594
Sample data: [['The', 'word', '``', 'atom', "''", 'was', 'coined', 'by', 'ancient', 'Greek', 'philosophers', '.'], ['However', ',', 'these', 'ideas', 'were', 'founded', 'in', 'philosophical', 'and', 'theological', 'reasoning', 'rather', 'than', 'evidence', 'and', 'experimentation', '.'], ['As', 'a', 'result', ',', 'their', 'views', 'on', 'what', 'atoms', 'look', 'like', 'and', 'how', 'they', 'behave', 'were', 'incorrect', '.'], ['They', 'also', 'could', 'not', 'convince', 'everybody', ',', 'so', 'atomism', 'was', 'but', 'one', 'of', 'a', 'number', 'of', 'competing', 'theories', 'on', 'the', 'nature', 'of', 'matter', '.'], ['It', 'was', 'not', 'until', 'the', '19th', 'century', 'that', 'the', 'idea', 'was', 'embraced', 'and', 'refined', 'by', 'scientists', ',', 'when', 'the', 'blossoming', 'science', 'of', 'chemistry', 'produced', 'discoveries', 'that', 'only', 'the', 'concept', 'of', 'atoms', 'could', 'explain', '.'], ['In', 'the', 'early', '1800s', 

In [ ]:
# Shuffle the sentences
shuffle(word_tokens)
print(word_tokens[0:5])

[['There', 'were', '244', 'households', 'out', 'of', 'which', '28.7', '%', 'had', 'children', 'under', 'the', 'age', 'of', '18', 'living', 'with', 'them', ',', '54.5', '%', 'were', 'married', 'couples', 'living', 'together', ',', '14.8', '%', 'had', 'a', 'female', 'householder', 'with', 'no', 'husband', 'present', ',', 'and', '27.9', '%', 'were', 'non-families', '.'], ['The', 'city', 'was', 'named', 'after', 'Mora', ',', 'Sweden', '.'], ['In', 'the', 'township', 'the', 'population', 'was', 'spread', 'out', 'with', '25.7', '%', 'under', 'the', 'age', 'of', '18', ',', '8.6', '%', 'from', '18', 'to', '24', ',', '20.0', '%', 'from', '25', 'to', '44', ',', '37.1', '%', 'from', '45', 'to', '64', ',', 'and', '8.6', '%', 'who', 'were', '65', 'years', 'of', 'age', 'or', 'older', '.'], ['Most', 'of', 'these', 'have', 'been', 'public', 'schools', 'of', 'The', 'Jefferson', 'County', 'School', 'System', 'which', 'was', 'founded', 'in', '1898', '.'], ['As', 'of', 'January', '2014', ',', 'Southwest',

In [ ]:
print(len(word_tokens))

761582


In [ ]:
# Split the sentences into 90% train set and 10% 
train_set = word_tokens[0:int(0.9*len(word_tokens))]
test_set = word_tokens[int(0.9*len(word_tokens)):]

#### 1 - Trigram Language Model

In [ ]:
# Extract frequency of each ngram from the training set 
def extractNgramFreq(train_set):
  unigram_freq = {}
  bigram_freq = {}
  trigram_freq = {}

  for sentence in train_set:
    for i in range(len(sentence)):
      unigram = sentence[i]
      if unigram in unigram_freq:
        unigram_freq[unigram] += 1
      else:
        unigram_freq[unigram] = 1
    
    for i in range(len(sentence)-1):
      bigram = (sentence[i], sentence[i+1])
      if bigram in bigram_freq:
        bigram_freq[bigram] += 1
      else:
        bigram_freq[bigram] = 1
    
    for i in range(len(sentence)-2):
      trigram = (sentence[i], sentence[i+1], sentence[i+2])
      if trigram in trigram_freq:
        trigram_freq[trigram] += 1
      else:
        trigram_freq[trigram] = 1
  
  return unigram_freq, bigram_freq, trigram_freq

# Get probability of each unigram based on frequency and total frequency
def getUnigramProb(unigram_freq):
  count = 0
  for unigram in unigram_freq:
    count += unigram_freq[unigram]
  
  unigram_prob = {}
  for unigram in unigram_freq:
    unigram_prob[unigram] = unigram_freq[unigram]/count
  
  return unigram_prob


# Get probability of each bigram based on frequency and total frequency
def getBigramProb(unigram_freq, bigram_freq):
  bigram_prob = {}
  for bigram in bigram_freq:
    bigram_prob[bigram] = bigram_freq[bigram]/unigram_freq[bigram[0]]
  
  return bigram_prob


# Get probability of each trigram based on frequency and total frequency
def getTrigramProb(bigram_freq, trigram_freq):
  trigram_prob = {}
  for trigram in trigram_freq:
    trigram_prob[trigram] = trigram_freq[trigram]/bigram_freq[(trigram[0], trigram[1])]
  
  return trigram_prob

##### Discounting method

In [ ]:
# Get discounted trigrams based on trigram and bigram frequency
def trigramsDiscounting(trigram_freq, bigram_freq, beta):
  discounted_trigrams = {}
  for trigram in trigram_freq:
    count = trigram_freq[trigram] - beta
    discounted_trigrams[trigram] = count / bigram_freq[(trigram[0], trigram[1])]
  
  return discounted_trigrams

# Calculate perplexity of the discounted trigrams 
def getDiscountingPerplexity(test_set, test_trigram_freq, discounted_trigrams):
  entropy = 0
  count = 0
  test_count = 0
  for trigram in test_trigram_freq:
    count += test_trigram_freq[trigram]
  
  for sentence in test_set:
    for i in range(len(sentence)-2):
      trigram = (sentence[i], sentence[i+1], sentence[i+2])
      if trigram in discounted_trigrams:
        prob = discounted_trigrams[trigram]
      else:
        prob = test_trigram_freq[trigram]/count
      
      if prob > 0:
        entropy += math.log(prob, 2)
      
      test_count += 1
  
  entropy = (-1*entropy)/test_count
  perplexity = math.pow(2, entropy)
  return perplexity

In [ ]:
# Discounting smoothing of trigrams
def discounting(train_set, test_set):
  unigram_freq, bigram_freq, trigram_freq = extractNgramFreq(train_set)
  unigram_prob = getUnigramProb(unigram_freq)
  bigram_prob = getBigramProb(unigram_freq, bigram_freq)
  trigram_prob = getTrigramProb(bigram_freq, trigram_freq)

  test_unigram_freq, test_bigram_freq, test_trigram_freq = extractNgramFreq(test_set)

  beta = round(uniform(0.6,0.7), 2)
  discounted_trigrams = trigramsDiscounting(trigram_freq, bigram_freq, beta)
  perplexity = getDiscountingPerplexity(test_set, test_trigram_freq, discounted_trigrams)
  return beta, perplexity

##### Interpolation method

In [ ]:
# Calculate lambda set for linear interpolation smoothing
def getLambda(test_set, epsilon, unigram_prob, bigram_prob, trigram_prob):
  lambda_set = [0.2, 0.3, 0.5]
  while True:
    exp_count = [0, 0, 0]
    for sentence in test_set:
      for i in range(len(sentence)-2):
        unigram = sentence[i]
        bigram = (sentence[i], sentence[i+1])
        trigram = (sentence[i], sentence[i+1], sentence[i+2])
        
        prob1 = unigram_prob[unigram] if unigram in unigram_prob else 0
        prob2 = bigram_prob[bigram] if bigram in bigram_prob else 0
        prob3 = trigram_prob[trigram] if trigram in trigram_prob else 0

        prob = lambda_set[0]*prob1 + lambda_set[1]*prob2 + lambda_set[2]*prob3

        if prob > 0:
          exp_count[0] += (lambda_set[0]*prob1)/prob
          exp_count[1] += (lambda_set[1]*prob2)/prob
          exp_count[2] += (lambda_set[2]*prob3)/prob
    
    exp_sum = sum(exp_count)
    if exp_sum == 0:
      return lambda_set
    
    new_lambda_set = [exp/exp_sum for exp in exp_count]

    flag = True
    for i in range(0, 3):
      diff = abs(new_lambda_set[i]-lambda_set[i])
      if diff > epsilon:
        flag = False
        break
    
    if flag:
      return lambda_set
    else:
      lambda_set = new_lambda_set


# Calculate perplexity for interpolation
def getInterpolationPerplexity(test_set, lambda_set, unigram_prob, bigram_prob, trigram_prob):
  entropy = 0
  test_count = 0
  for sentence in test_set:
    for i in range(len(sentence)-2):
      unigram = sentence[i]
      bigram = (sentence[i], sentence[i+1])
      trigram = (sentence[i], sentence[i+1], sentence[i+2])
        
      prob1 = unigram_prob[unigram] if unigram in unigram_prob else 0
      prob2 = bigram_prob[bigram] if bigram in bigram_prob else 0
      prob3 = trigram_prob[trigram] if trigram in trigram_prob else 0

      prob = lambda_set[0]*prob1 + lambda_set[1]*prob2 + lambda_set[2]*prob3
      if prob > 0:
        entropy += math.log(prob, 2)
      
      test_count += 1
  
  entropy = (-1*entropy)/test_count
  perplexity = math.pow(2, entropy)
  return perplexity

In [ ]:
# Linear Interpolation smoothing
def interpolation(train_set, test_set):
  unigram_freq, bigram_freq, trigram_freq = extractNgramFreq(train_set)
  unigram_prob = getUnigramProb(unigram_freq)
  bigram_prob = getBigramProb(unigram_freq, bigram_freq)
  trigram_prob = getTrigramProb(bigram_freq, trigram_freq)

  lambda_set = getLambda(test_set, 0.005, unigram_prob, bigram_prob, trigram_prob)
  perplexity = getInterpolationPerplexity(test_set, lambda_set, unigram_prob, bigram_prob, trigram_prob)
  return lambda_set, perplexity

#### 2 - Splitting training dataset

In [ ]:
# Split original training set into training set and validation set
def splitTrainSet(train_set):
  shuffle(train_set)
  new_train_set = train_set[0:int(len(train_set)*0.9)]
  validation_set = train_set[int(len(train_set)*0.9):]
  return new_train_set, validation_set

In [ ]:
ts1, vs1 = splitTrainSet(train_set)
ts2, vs2 = splitTrainSet(train_set)
ts3, vs3 = splitTrainSet(train_set)
ts4, vs4 = splitTrainSet(train_set)
ts5, vs5 = splitTrainSet(train_set)

#### 3 - Performance on Validation Set

##### Discounting

In [ ]:
# Calculate discounting beta and perplexity for all validation sets
beta1, perp1 = discounting(ts1, vs1)
beta2, perp2 = discounting(ts2, vs2)
beta3, perp3 = discounting(ts3, vs3)
beta4, perp4 = discounting(ts4, vs4)
beta5, perp5 = discounting(ts5, vs5)
beta = [beta1, beta2, beta3, beta4, beta5]
perp = [perp1, perp2, perp3, perp4, perp5]

print('Discounting performance on validation set')
for i in range(0, 5):
  print('Set', i+1, '- beta:', beta[i], ', perplexity:', perp[i])

Discounting performance on validation set
Set 1 - beta: 0.61 , perplexity: 385.98464393175266
Set 2 - beta: 0.61 , perplexity: 335.20801152217865
Set 3 - beta: 0.66 , perplexity: 362.5238493150579
Set 4 - beta: 0.64 , perplexity: 378.2673871942632
Set 5 - beta: 0.62 , perplexity: 346.0446900001597


##### Interpolation

In [ ]:
# Calculate interpolation lambda and perplexity for all validation sets
ls1, perp1 = interpolation(ts1, vs1)
ls2, perp2 = interpolation(ts2, vs2)
ls3, perp3 = interpolation(ts3, vs3)
ls4, perp4 = interpolation(ts4, vs4)
ls5, perp5 = interpolation(ts5, vs5)
ls = [ls1, ls2, ls3, ls4, ls5]
perp = [perp1, perp2, perp3, perp4, perp5]

print('Interpolation performance on validation set')
for i in range(0, 5):
  print('Set', i+1, '- lambda set:', ls[i], ', perplexity:', perp[i])

Interpolation performance on validation set
Set 1 - lambda set: [0.3277119588920003, 0.39650946907696855, 0.2757785720310312] , perplexity: 91.22769316275311
Set 2 - lambda set: [0.3295335779794422, 0.3963021384365197, 0.2741642835840382] , perplexity: 92.42521601363109
Set 3 - lambda set: [0.32792929962145495, 0.396991874037794, 0.2750788263407509] , perplexity: 91.77695155824924
Set 4 - lambda set: [0.3307374088539322, 0.39644097666968764, 0.2728216144763801] , perplexity: 93.04468779624617
Set 5 - lambda set: [0.3294072328392538, 0.3964805020221355, 0.27411226513861076] , perplexity: 92.3473527959448


#### 4 - Performance on Test Set

##### Discounting

In [ ]:
# Calculate discounting beta and perplexity for all test sets and compute variance
beta1, perp1 = discounting(ts1, test_set)
beta2, perp2 = discounting(ts2, test_set)
beta3, perp3 = discounting(ts3, test_set)
beta4, perp4 = discounting(ts4, test_set)
beta5, perp5 = discounting(ts5, test_set)
beta = [beta1, beta2, beta3, beta4, beta5]
perp = [perp1, perp2, perp3, perp4, perp5]

print('Discounting performance on test set')
for i in range(0, 5):
  print('Set', i+1, '- beta:', beta[i], ', perplexity:', perp[i])

print("Variance of perplexity:", statistics.variance(perp))

Discounting performance on test set
Set 1 - beta: 0.65 , perplexity: 384.1294669832734
Set 2 - beta: 0.61 , perplexity: 379.87093784436684
Set 3 - beta: 0.63 , perplexity: 383.77810953043127
Set 4 - beta: 0.64 , perplexity: 382.5792679308408
Set 5 - beta: 0.64 , perplexity: 382.0304207051159
Variance of perplexity: 2.8562457111186608


##### Interpolation

In [ ]:
# Calculate interpolation lambda and perplexity for all test sets and compute variance
ls1, perp1 = interpolation(ts1, test_set)
ls2, perp2 = interpolation(ts2, test_set)
ls3, perp3 = interpolation(ts3, test_set)
ls4, perp4 = interpolation(ts4, test_set)
ls5, perp5 = interpolation(ts5, test_set)
ls = [ls1, ls2, ls3, ls4, ls5]
perp = [perp1, perp2, perp3, perp4, perp5]

print('Interpolation performance on test set')
for i in range(0, 5):
  print('Set', i+1, '- lambda set:', ls[i], ', perplexity:', perp[i])

print("Variance of perplexity:", statistics.variance(perp))

Interpolation performance on test set
Set 1 - lambda set: [0.3296833397486049, 0.39652110232477983, 0.27379555792661514] , perplexity: 91.99739879716303
Set 2 - lambda set: [0.3295044469515101, 0.39658843144569367, 0.2739071216027961] , perplexity: 91.78268638383216
Set 3 - lambda set: [0.3296018985447135, 0.39650282503347606, 0.2738952764218105] , perplexity: 91.87850940426299
Set 4 - lambda set: [0.3296188623024507, 0.3963528913693927, 0.27402824632815653] , perplexity: 91.91424649511205
Set 5 - lambda set: [0.3294907874206512, 0.3965074778124422, 0.27400173476690654] , perplexity: 91.8246370330346
Variance of perplexity: 0.006872811974727759


#### 5 - Laplace Smoothing

In [ ]:
# Get laplace probability of trigrams based on trigram and bigram frequency
def getLaplaceProb(trigram_freq, bigram_freq):
  k = 1
  trigram_prob = {}
  for trigram in trigram_freq:
    bigram = (trigram[0], trigram[1])
    trigram_prob[trigram] = (trigram_freq[trigram] + k) / (bigram_freq[bigram]+ k*len(trigram_freq))
  return trigram_prob


# Calculate perplexity of laplace smoothing
def getLaplacePerplexity(test_set, trigram_prob):
  entropy = 0
  test_count = 0
  for sentence in test_set:
    for i in range(len(sentence)-2):
      trigram = (sentence[i], sentence[i+1], sentence[i+2])
      prob = trigram_prob[trigram] if trigram in trigram_prob else 0

      if prob > 0:
        entropy += math.log(prob, 2)

      test_count += 1

  entropy = (-1*entropy)/test_count
  perplexity = math.pow(2, entropy)
  return perplexity

In [ ]:
# Laplace smoothing
def laplace(train_set, test_set):
  unigram_freq, bigram_freq, trigram_freq = extractNgramFreq(train_set)
  trigram_prob = getLaplaceProb(trigram_freq, bigram_freq)
  perplexity = getLaplacePerplexity(test_set, trigram_prob)
  return perplexity

In [ ]:
# Calculate laplace perplexity for all test sets and compute variance
perp1 = laplace(ts1, test_set)
perp2 = laplace(ts2, test_set)
perp3 = laplace(ts3, test_set)
perp4 = laplace(ts4, test_set)
perp5 = laplace(ts5, test_set)
perp = [perp1, perp2, perp3, perp4, perp5]

print('Laplace performance on test set')
for i in range(0, 5):
  print('Set', i+1, '- perplexity:', perp[i])

print("Variance of perplexity:", statistics.variance(perp))

Laplace performance on test set
Set 1 - perplexity: 589.2159070170205
Set 2 - perplexity: 589.9118492378913
Set 3 - perplexity: 588.7354057333338
Set 4 - perplexity: 591.6856934734319
Set 5 - perplexity: 589.5165635032213
Variance of perplexity: 1.2805890031643805


### 2.2 - Vector Semantics: GloVE implementation

#### 1 - GloVe embedding method implementation

In [ ]:
# Import required libraries
from collections import Counter
from scipy import sparse
import numpy as np
from math import log
from tqdm import tqdm
import pickle

In [ ]:
# Builds a vocabulary of words mapped to word ID and word frequency in the corpus
def build_vocabulary(corpus):
  vocabulary = Counter()
  # Tokenize corpus into sentences
  sentences = sent_tokenize(corpus)
  for sentence in sentences:
    # Tokenize each sentence into words and update the vocabulary
    words = word_tokenize(sentence)
    vocabulary.update(words)
  
  # Return vocabulary and list of sentences
  return {word: (id, freq) for id, (word, freq) in enumerate(vocabulary.items())}, sentences

In [ ]:
# Builds the word cooccurrence matrix X for the given vocabulary. The final
# result is a list containing tuples of (center_ID, context_ID, Xij)
def build_cooccurrences(vocabulary, sentences, contextWindow):
  # Let V be the size of the vocabulary
  vocabSize = len(vocabulary)
  result = []

  # Initialize a sparse matrix for cooccurrences of size V x V
  cooccurrences = sparse.lil_matrix((vocabSize, vocabSize), dtype=np.float64)

  # Iterate over all sentences in the corpus
  for i, sentence in enumerate(sentences):
    # Iterate over all words in the sentence and fetch their word IDs in a list
    words = word_tokenize(sentence)
    wordIds = [vocabulary[word][0] for word in words]

    for i, centerId in enumerate(wordIds):
      # Collect all context word IDs in left window of center word
      contextIds = wordIds[max(0, i - contextWindow) : i]
      contextLen = len(contextIds)

      # Iterate over each context word in left window
      for j, contextId in enumerate(contextIds):
        # Distance from center word
        distance = contextLen - j
        
        # Weight by inverse of distance between words
        increment = 1.0 / float(distance)
        
        # Update cooccurrences of context word and center word symmetrically to
        # handle both left window and right window cooccurrences
        cooccurrences[centerId, contextId] += increment
        cooccurrences[contextId, centerId] += increment
  
  # Build the output result from the sparse matrix
  for i, (row, data) in enumerate(zip(cooccurrences.rows, cooccurrences.data)):
    for data_idx, j in enumerate(row):
      result.append((i, j, data[data_idx]))
  
  return result

In [ ]:
# Runs a single iteration of adaptive gradient descent (AdaGrad) while training
# the GloVe word embeddings. Takes input cooccurrence data, weight vectors,
# biases and gradient histories. Returns the cost associated with
# the given weights and updates the weights by AdaGrad
def run_iteration(vocabulary, data, learningRate, xMax, alpha):
  globalCost = 0

  # Shuffle the data to avoid biasing of the word vectors
  shuffle(data)

  for (vMain, vContext, bMain, bContext, gradWMain, gradWContext,
       gradBMain, gradBContext, cooccurrence) in data:
    
    weight = (cooccurrence / xMax) ** alpha if cooccurrence < xMax else 1
    
    # Compute inner component of cost function
    costInner = (vMain.dot(vContext) + bMain[0] + bContext[0] - log(cooccurrence))
    
    # Compute cost function
    cost = weight * (costInner ** 2)

    # Add weighted cost to the global cost
    globalCost += 0.5 * cost

    # Compute gradients for word vectors
    gradMain = weight * costInner * vContext
    gradContext = weight * costInner * vMain

    # Compute gradients for bias terms
    gradBiasMain = weight * costInner
    gradBiasContext = weight * costInner

    # Perform adaptive gradient descent
    vMain -= (learningRate * gradMain / np.sqrt(gradWMain))
    vContext -= (learningRate * gradContext / np.sqrt(gradWContext))

    bMain -= (learningRate * gradBiasMain / np.sqrt(gradBMain))
    bContext -= (learningRate * gradBiasContext / np.sqrt(gradBContext))
    
    # Update squared gradient sums
    gradWMain += np.square(gradMain)
    gradWContext += np.square(gradContext)
    gradBMain += gradBiasMain ** 2
    gradBContext += gradBiasContext ** 2
  
  return globalCost

In [ ]:
# Train GloVe vectors given cooccurrences and vocabulary. Takes input other
# parameters such as dimSize, iterations, learningRate, xMax and alpha. Returns
# the computed word vector matrix W of size 2V * d
def train_glove(vocabulary, cooccurrences, dimSize, iterations, learningRate, xMax, alpha):
  
  vocabSize = len(vocabulary)

  # Word vector matrix of size 2V * d initialized by random values in range
  # (-0.5, 0.5]
  W = (np.random.rand(vocabSize * 2, dimSize) - 0.5) / float(dimSize + 1)

  # Bias terms associated with each single vector initialized by random values
  # in range (-0.5, 0.5]
  biases = (np.random.rand(vocabSize * 2) - 0.5) / float(dimSize + 1)

  # Sum of squares of all previous gradients for adaptive gradient descent
  # (AdaGrad) initialized to 1 so that initial adaptive learning rate is equal
  # to global learning rate
  gradient = np.ones((vocabSize * 2, dimSize), dtype=np.float64)

  # Sum of squared gradients for the bias terms
  gradientBiases = np.ones(vocabSize * 2, dtype=np.float64)

  data = [ (W[iMain], W[iContext + vocabSize],
            biases[iMain : iMain + 1],
            biases[iContext + vocabSize : iContext + vocabSize + 1],
            gradient[iMain], gradient[iContext + vocabSize],
            gradientBiases[iMain : iMain + 1],
            gradientBiases[iContext + vocabSize : iContext + vocabSize + 1],
            cooccurrence )
            for iMain, iContext, cooccurrence in cooccurrences]
  
  # Train the word vector matrix for specific number of iterations
  for i in tqdm(range(iterations)):
    cost = run_iteration(vocabulary, data, learningRate, xMax, alpha)
    print('Iteration', i, '- Cost:', cost)

  # Return the word vector matrix
  return W

In [ ]:
def glove_embeddings(corpus, contextWindow, dimSize, iterations, learningRate, xMax, alpha):
  vocabulary, sentences = build_vocabulary(corpus)
  cooccurrences = build_cooccurrences(vocabulary, sentences, contextWindow)
  W = train_glove(vocabulary, cooccurrences, dimSize, iterations, learningRate, xMax, alpha)
  return W, vocabulary

In [ ]:
CONTEXT_WINDOW = 10
DIM_SIZE = 100
ITERATIONS = 50
LEARNING_RATE = 0.05
X_MAX = 100
ALPHA = 0.75
SMALL_RATIO = 0.15
CORPUS = eng_text[0:int(SMALL_RATIO*len(eng_text))]

W, vocabulary = glove_embeddings(CORPUS, contextWindow=CONTEXT_WINDOW, dimSize=DIM_SIZE,
                            iterations=ITERATIONS, learningRate=LEARNING_RATE,
                            xMax=X_MAX, alpha=ALPHA)

print('Vocabulary Size:', len(vocabulary))

  2%|▏         | 1/50 [05:05<4:09:47, 305.87s/it]

Iteration 0 - Cost: 246414.4416228635


  4%|▍         | 2/50 [10:08<4:03:57, 304.94s/it]

Iteration 1 - Cost: 154526.7653301233


  6%|▌         | 3/50 [15:10<3:58:04, 303.93s/it]

...

 98%|█████████▊| 49/50 [4:06:06<04:57, 297.75s/it]

Iteration 48 - Cost: 40415.34033631018


100%|██████████| 50/50 [4:11:06<00:00, 301.34s/it]

Iteration 49 - Cost: 39985.61136303595


Vocabulary Size: 99287


In [ ]:
# Final GloVe word embedding is obtained by adding the center word embedding
# and context word embedding for each word
print(W.shape)
embeddings = {}
vocabSize = len(vocabulary)
for word, (id, _) in vocabulary.items():
  embeddings[word] = W[id]+W[id+vocabSize]

# Save the embeddings in a pickle file
with open('embeddings.pickle', 'wb') as handle:
    pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the weights in a pickle file
with open('weights.pickle', 'wb') as handle:
    pickle.dump(W, handle, protocol=pickle.HIGHEST_PROTOCOL)

(153650, 100)


#### 2 - Comparing Word Similarity
Dependencies: embeddings.pickle

In [ ]:
# Load the embeddings from pickle file
with open('embeddings.pickle', 'rb') as handle:
    embeddings = pickle.load(handle)

In [ ]:
# Copy the benchmark tool to the current session
! rm -rf web
! cp -r '/content/drive/My Drive/CS565/Assignment 2/word-embeddings-benchmarks/web' .

In [ ]:
# Import required libraries
from web.datasets.similarity import *
from web.evaluate import evaluate_similarity
from web.embeddings import fetch_GloVe

In [ ]:
# Define benchmark tasks for word similarity comparison 
tasks = {
    "MEN": fetch_MEN(),
    "WS353": fetch_WS353(),
    "SIMLEX999": fetch_SimLex999(),
    "RG65": fetch_RG65(),
    "MTurk": fetch_MTurk(),
    "RW": fetch_RW()
}

In [ ]:
# Print sample data
for name, data in tasks.items():
    print("Sample data from {}: pair \"{}\" and \"{}\" is assigned score {}".format(name, data.X[0][0], data.X[0][1], data.y[0]))

Sample data from MEN: pair "sun" and "sunlight" is assigned score [10.]
Sample data from WS353: pair "love" and "sex" is assigned score 6.77
Sample data from SIMLEX999: pair "old" and "new" is assigned score 1.58
Sample data from RG65: pair "gem" and "jewel" is assigned score 9.85
Sample data from MTurk: pair "episcopal" and "russia" is assigned score 5.5
Sample data from RW: pair "squishing" and "squirt" is assigned score 5.88


In [ ]:
# Calculate results using helper function
for name, data in tasks.items():
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(embeddings, data.X, data.y)))

Missing 803 words. Will replace them with mean vector


Spearman correlation of scores on MEN 0.1295455136075063


Missing 52 words. Will replace them with mean vector


Spearman correlation of scores on WS353 0.20866820523498564


Missing 94 words. Will replace them with mean vector


...

Spearman correlation of scores on RG65 0.1629543816722517


Missing 125 words. Will replace them with mean vector


Spearman correlation of scores on MTurk 0.18482107742423962


Missing 1743 words. Will replace them with mean vector


Spearman correlation of scores on RW 0.20551540968104426


In [ ]:
# Fetch GloVe embeddings
gloveEmbeddings = fetch_GloVe(corpus="twitter-27B", dim=100)

File already downloaded, skipping


In [ ]:
# Calculate results using helper function
for name, data in tasks.items():
    print("Spearman correlation of scores on {} {}".format(name, evaluate_similarity(gloveEmbeddings, data.X, data.y)))

Missing 1 words. Will replace them with mean vector
Missing 25 words. Will replace them with mean vector
Missing 1 words. Will replace them with mean vector
Missing 1 words. Will replace them with mean vector


Spearman correlation of scores on MEN 0.5773369776281995
Spearman correlation of scores on WS353 0.46979381939437287
Spearman correlation of scores on SIMLEX999 0.12221121100798378
Spearman correlation of scores on RG65 0.6774486160113895


Missing 1071 words. Will replace them with mean vector


Spearman correlation of scores on MTurk 0.5641004632190647
Spearman correlation of scores on RW 0.23074174522387267
